Формулировка задачи:

Необходимо создать консольную (работающую из командной строки) утилиту. Программа спрашивает домен и забирает с введённого адреса файл robots.txt. Затем скрипт парсит файл и выводит его содержимое в виде объекта. Ключами объекта являются параметры User-Agent, а значениями — вложенный объект. Вложенный объект содержит два поля Allow и Dissallow, каждый из которых является массивом соответствующих URL из robots.txt. Код выложить на GitHub
Пример:

{
  "*": {
      "Disallow": ["/cgi-bin"],
      "Allow": ["/"]
   },
  "GoogleBot": {
      "Disallow": ["/cgi-bin"],
      "Allow": ["/"]
   }
}

Декомпозиция задачи:
1. У нас консольная программа, но передавать параметры при запуске вроде бы не нужно. Есть слово "скрипт", но нет ограничения на язык. Пусть будет python, его (при прочих равных) приятно читать и видеть. Системой пусть будет linux, но ничего платформоспецифичного вроде бы не предполагается.

2. Нужно спросить о домене и забрать с введённого адреса. Предполагаю, что "домен" и "адрес" здесь понимаются как синонимы, а не как "домен + адрес" или как-то ещё. Есть два способа это сделать - написать текстом строку и вбить IP-адрес. Можно поставить проверки на основе регулярных выражений, которые покажут, является ли введённый текст чем-то осмысленным. Ещё можно ругаться, если нет такого адреса, но для этого вроде бы есть готовая ругалка

3. Нужно вытащить файл и прочитать. Возможные проблемы: по адресу нет файла, нет нужной структуры внутри.

4. Чтобы вернуть что нужно, можно было бы определить пользовательский объект. Но вид объекта наводит на мысль либо о словаре словарей, который уже реализован. Воспользуемся им.

5. В требованиях есть слово "выводит содержимое в виде объекта". По некоторому размышлению, предполагает, что мы, возможно, хотим уметь сериализовать (или хотя бы выводить на печать) полученный объект, чтобы прямо в консоли видеть результат. С этим тоже нет проблемы, но для красивого вывода можно пробежаться по ключам, чтобы не смотреть совсем уж на мешанину.

Методы:

1. Получить url
2. Соединиться
3. Распарсить
4. Прибраться за собой


Замечу, что на stackoverflow <a href = https://stackoverflow.com/questions/43085744/parsing-robots-txt-in-python#43086135> есть </a> наводящее на нужные мысли (правда, для python 2) решение такого вот вида:

In [1]:
import os
result = os.popen("curl https://fortune.com/robots.txt").read()
result_data_set = {"Disallowed":[], "Allowed":[]}

for line in result.split("\n"):
    if line.startswith('Allow'):    # this is for allowed url
        result_data_set["Allowed"].append(line.split(': ')[1].split(' ')[0])    # to neglect the comments or other junk info
    elif line.startswith('Disallow'):    # this is for disallowed url
        result_data_set["Disallowed"].append(line.split(': ')[1].split(' ')[0])    # to neglect the comments or other junk info

print (result_data_set)

{'Allowed': ['/wp-admin/admin-ajax.php'], 'Disallowed': ['/wp-admin/', '/search/', '/wp-login.php', '/activate/', '/cgi-bin/', '/mshots/v1/', '/next/', '/public.api/']}


Кроме того, оно не учитывает наличие нескольких user-agent.

Дополним его и избавим от указанных проблем, попутно придав формат тестового задания.

In [190]:
import requests, os, re
# получим этот url от пользователя потом

def is_string_a_valid_url(string):
    # проверка корректности на случай, если это адрес, идея отсюда: 
    # https://web.izjum.com/regexp-email-url-phone
    url_regexp = '^((https?|ftp)\:\/\/)?([a-z0-9]{1})((\.[a-z0-9-])|([a-z0-9-]))*\.([a-z]{2,6})(\/?)$'
    if re.match(url_regexp, string) != None:
        return True
    else: 
        return False

def is_string_a_valid_ip(string):
    # и на случай, если это IP-адрес.
    # выражение - отсюда https://a-panov.ru/регулярные-выражения-проверка-ip-на-кор/
    ip_regexp = '/^(25[0-5]|2[0-4][0-9]|[0-1][0-9]{2}|[0-9]{2}|[0-9])(\.(25[0-5]|2[0-4][0-9]|[0-1][0-9]{2}|[0-9]{2}|[0-9])){3}$'
    if re.match(ip_regexp, string) != None:
        return True
    else: 
        return False

def ask_for_url():
    url = input("Enter the url you wich to retrieve 'robots.txt' from: ")
    
    # если это не цифры, и не начали с описания протокола, добавим по умолчанию https
    if ((not url.startswith('http')) and (not url[0].isdigit())):
        url = "https://" + url
    
    if (is_string_a_valid_url(url) == True) or (is_string_a_valid_ip(url) == True):
        url += '/robots.txt'
        return url
    else:
        print("You're probably entering not a valid url. Please make sure the input is correct.")
        return 1


def download_robots_txt(url, my_user_agent = '*'):
    # мы не знаем, какой/какие user-agent будем использовать, так что подадим его пока явно
    local_filename = "robots_local_temp.txt"
    my_headers = requests.utils.default_headers()
    my_headers.update({'User-Agent': my_user_agent})
    r = requests.get(url, headers = my_headers)
    # будем качать по частям, на случай, если файл очень велик
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    logstring = 'download finished, temporary file ./' + local_filename + ' created' 
    print(logstring)
    return 0

# избавимся от файла, чтобы он не болтался в каталоге
def dispose_of_temp_file():
    if os.path.exists("robots_local_temp.txt"):
        os.remove("robots_local_temp.txt")
        print('cleaning up...')
    return 0

# распарсим имеющийся файл
def parse_robots_txt():
    if os.path.exists("robots_local_temp.txt"):
        local_file = open('robots_local_temp.txt', 'r')
        resulting_data_set = {}
        data_set_for_user_agent = {"Disallowed":[], "Allowed":[]}
        agent_name_current = 'NoAgentYet'
        for line in local_file:
            line = line.split('\n')[0]
            if line.startswith('User-agent'):
                agent_name_next = line.split(': ')[1].split(' ')[0]
                # пока первого агента нет, будет пустое поле в словаре. Зато после изменения агента мы 
                # запишем все собранные сведения
                # сотрём накопленное и продолжим наполнять словарь.
                if not agent_name_current == 'NoAgentYet':
                    resulting_data_set[agent_name_current] = data_set_for_current_user_agent
                agent_name_current = agent_name_next
                data_set_for_current_user_agent = {"Disallowed":[], "Allowed":[]}
            else:
                if line.startswith('Allow'):
                    # комментарии и мусор немного отфильтруем
                    data_set_for_current_user_agent["Allowed"].append(line.split(': ')[1].split(' ')[0])    
                elif line.startswith('Disallow'):
                    # аналогично
                    data_set_for_current_user_agent["Disallowed"].append(line.split(': ')[1].split(' ')[0])
        #после выхода из цикла мы останемся со сведениями о последнем (возможно, единственном) агенте
        agent_name_current=agent_name_next
        resulting_data_set[agent_name_current] = data_set_for_current_user_agent
        return resulting_data_set
    else:
        print('A local copy of robots.txt is missing for some reason.')
        return 1
    
def print_out_the_result():
    result = parse_robots_txt()
    if result == 1:
        print("No output created")
        return 1
    else:
        for key in result.keys():
            print(key, ':')
            for inner_key in result[key]:
                print(inner_key, ':')
                print(result[key][inner_key])
            print()
        return 0

In [183]:
dispose_of_temp_file()

cleaning up...


0

In [184]:
url = ask_for_url()
print(url)

Enter the url you wich to retrieve 'robots.txt' from: habrahabr.ru
https://habrahabr.ru/robots.txt


In [185]:
download_robots_txt(url)

download finished, temporary file ./robots_local_temp.txt created


0

In [186]:
result = parse_robots_txt()
print(result)


{'Yandex': {'Allowed': [], 'Disallowed': ['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']}, '*': {'Allowed': [], 'Disallowed': ['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']}, 'Slurp': {'Allowed': [], 'Disallowed': ['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']}, 'Googlebot': {'Allowed': [], 'Disallowed': ['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']}}


In [191]:
print_out_the_result()

Yandex :
Allowed :
[]
Disallowed :
['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']

* :
Allowed :
[]
Disallowed :
['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']

Slurp :
Allowed :
[]
Disallowed :
['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']

Googlebot :
Allowed :
[]
Disallowed :
['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']



0

In [175]:
dispose_of_temp_file()

cleaning up...


0